# Day 14: Regolith Reservoir

The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads behind the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly air with structures made of rock.

Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:

```
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
```

This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

In [3]:
EXAMPLE = """
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9"""

In [17]:
import numpy as np

In [108]:
ROCK = "#"
SAND = "o"
AIR = "."
START = "+"
START_POS = (500, 0)

def read_puzzle(s):
    m = {START_POS: START}
    for line in s.splitlines():
        parts = line.split(" -> ")
        for p0, p1 in zip(parts, parts[1:]):
            insert_rocks(m, p0, p1)
    return m

def pm_range(n):
    # The +1 is so that in the case of x0 == x1 we get dx in [0]
    if n >= 0:
        return range(n + 1)
    else:
        return range(0, n - 1, -1)
            
def insert_rocks(m, p0, p1):
    y0, x0 = [int(s) for s in p0.split(",")]
    y1, x1 = [int(s) for s in p1.split(",")]
    # This double loop gives only the correct result if there are
    # no diagonal paths. 
    for dx in pm_range(x1 - x0):
        for dy in pm_range(y1 - y0):
            m[y0 + dy, x0 + dx] = ROCK

In [145]:
class Cave:
    def __init__(self, d):
        self.d = d
        self.dims = ((self.ymin, self.ymax), (self.xmin, self.xmax)) = self.__dimensions()
        
    def get(self, pos):
        return self.d.get(pos, AIR)
    
    def set(self, pos, val):
        assert self.contains(pos)
        self.d[pos] = val
    
    def contains(self, pos):
        """See if pos is inside cave.
        """
        y, x = pos
        return (self.ymin <= y <= self.ymax) and (self.xmin <= x <= self.xmax)
        
    def __dimensions(self):
        ys, xs = zip(*self.d.keys()) # thanks to https://stackoverflow.com/a/19343/20773725
        ymin, ymax = min(ys), max(ys)
        xmin, xmax = min(xs), max(xs)
        return ((ymin, ymax), (xmin, xmax))

In [166]:
m = Cave(read_puzzle(EXAMPLE))

In [167]:
def print_maze(m):
    ((ymin, ymax), (xmin, xmax)) = m.dims
    for x in range(xmin, xmax + 1):
        for y in range(ymin, ymax + 1):
            val = m.get((y, x))
            if val == ROCK:
                print("#", end="")
            elif val == SAND:
                print("o", end="")
            elif val == START:
                print("+", end="")                
            else:
                print(".", end="")
        print()

In [168]:
print_maze(m)

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
........#.
#########.


The sand is pouring into the cave from point 500,0.

Drawing rock as #, air as ., and the source of the sand as +, this becomes:

```
  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
```

Sand is produced one unit at a time, and the next unit of sand is not produced until the previous unit of sand comes to rest. A unit of sand is large enough to fill one tile of air in your scan.

A unit of sand always falls down one step if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally one step down and to the left. If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves, at which point the next unit of sand is created back at the source.

In [169]:
def fall_sand(m):
    """One unit of sand starts falling from START_POS.
    Modifies *m*.
    Returns True if unit of sand could be deposits, False otherwise.
    """
    (y, x) = START_POS
    while True:
        # Try all three directions
        for p in [(y, x+1), (y-1, x+1), (y+1, x+1)]:
            val = m.get(p)
            if not m.contains(p):
                # outside of cave this choice is taken and the unit enters the void
                break
            elif val == AIR:
                # Valid move -> try next round
                break
            elif val in (ROCK, SAND):
                # try next move
                continue
            else:
                raise Exception("What happens at (%d, %d)?" % (y, x))

        if not m.contains(p):
            return False
        elif m.get(p) == AIR:
            # Valid move -> try next round
            (y, x) = p
            continue
        else:
            # Could not find a valid move. Deposit unit of sand at last pos
            m.set((y, x), SAND)
            return True

So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.
```

The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
```

After a total of five units of sand have come to rest, they form this pattern:

```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
```

After a total of 22 units of sand:

```
......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
```

In [175]:
while fall_sand(m):
    pass

print_maze(m)

......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.


Finally, only two more units of sand can possibly come to rest:

```
......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.
```

Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:

```
.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
```

Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?


In [181]:
len([x for x in m.d.values() if x == SAND])

24

Putting this all in one function:

In [182]:
def part1(s):
    m = Cave(read_puzzle(s))
    while fall_sand(m):
        pass

    print_maze(m)
    return len([x for x in m.d.values() if x == SAND])

In [183]:
part1(EXAMPLE)

......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.


24

In [184]:
with open("input") as f:
    input_maze = f.read()

In [186]:
part1(input_maze)

...........................+...........................
.......................................................
.......................................................
.......................................................
.......................................................
.......................................................
.......................................................
.......................................................
.......................................................
.......................................................
...........................o...........................
..........................ooo..........................
.........................ooooo.........................
........................#ooo#oo........................
........................#ooo#ooo.......................
........................#ooo#oooo......................
.......................o#ooo#ooooo.....................
......................###ooo#########...........

618